In [1]:
# !pip install -r requirements.txt
!pip install rake-nltk
!wget https://raw.githubusercontent.com/bangkitjkt/bangkit-1/master/data/netflix_titles.csv

--2020-04-05 12:37:35--  https://raw.githubusercontent.com/bangkitjkt/bangkit-1/master/data/netflix_titles.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2410660 (2.3M) [text/plain]
Saving to: ‘netflix_titles.csv.2’

netflix_titles.csv. 100%[===================>]   2.30M  --.-KB/s    in 0.09s   

2020-04-05 12:37:35 (25.9 MB/s) - ‘netflix_titles.csv.2’ saved [2410660/2410660]



In [0]:
# Import dependencies
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf

In [3]:
# read dataset file
df = pd.read_csv('netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [4]:
# select only the features
df = df[['title', 'director', 'cast', 'rating', 'listed_in', 'description']]
df.head()

,title,director,cast,rating,listed_in,description
0,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...",TV-PG,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,Jandino: Whatever it Takes,NaN,Jandino Asporaat,TV-MA,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",TV-Y7-FV,Kids' TV,"With the help of three human allies, the Autob..."
3,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",TV-Y7,Kids' TV,When a prison ship crash unleashes hundreds of...
4,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",TV-14,Comedies,When nerdy high schooler Dani finally attracts...


In [5]:
# fill the null
df['director']=df['director'].fillna(' ')
df['director']=df['director'].astype('str')
df['cast']=df['cast'].fillna(' ')
df['cast']=df['cast'].astype('str')
df.dropna(inplace=True)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6224 entries, 0 to 6233
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        6224 non-null   object
 1   director     6224 non-null   object
 2   cast         6224 non-null   object
 3   rating       6224 non-null   object
 4   listed_in    6224 non-null   object
 5   description  6224 non-null   object
dtypes: object(6)
memory usage: 340.4+ KB


,title,director,cast,rating,listed_in,description
0,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...",TV-PG,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,Jandino: Whatever it Takes,,Jandino Asporaat,TV-MA,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,Transformers Prime,,"Peter Cullen, Sumalee Montano, Frank Welker, J...",TV-Y7-FV,Kids' TV,"With the help of three human allies, the Autob..."
3,Transformers: Robots in Disguise,,"Will Friedle, Darren Criss, Constance Zimmer, ...",TV-Y7,Kids' TV,When a prison ship crash unleashes hundreds of...
4,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",TV-14,Comedies,When nerdy high schooler Dani finally attracts...


In [0]:
# initializing the new column
df['bag_of_words'] = ''
for index, row in df.iterrows():
    plot = row['description']
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    g = ''.join(row['listed_in'].split(',')).lower()
    d = ''.join(row['director'].replace(' ','').split(',')).lower()
    a = ' '.join(row['cast'].replace(' ','').split(',')).lower()
    r = ' '.join(row['rating']).lower()
    k = ' '.join(list(key_words_dict_scores.keys()))
    row['bag_of_words'] = g + ' ' + ' ' + d + ' ' + a + ' ' + r + ' ' + k

newdf = df[['title', 'bag_of_words']]

In [7]:
newdf.head()

,title,bag_of_words
0,Norm of the North: King Sized Adventure,children & family movies comedies richardfinn...
1,Jandino: Whatever it Takes,stand-up comedy jandinoasporaat t v - m a ra...
2,Transformers Prime,kids' tv petercullen sumaleemontano frankwel...
3,Transformers: Robots in Disguise,kids' tv willfriedle darrencriss constancezi...
4,#realityhigh,comedies fernandolebrija nestacooper katewals...


In [0]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(newdf['title'])

In [0]:
# compute cosine_distance manually
c = CountVectorizer()
def compute_cosine_distances(a, b):
    normalize_a = tf.nn.l2_normalize(a,1)        
    normalize_b = tf.nn.l2_normalize(b,1)
    distance = tf.matmul(normalize_a, normalize_b, transpose_b=True)
    return distance

count_mat_arr = c.fit_transform(newdf['bag_of_words']).toarray().astype(np.float64)

tf_cosine_sim = compute_cosine_distances(count_mat_arr, count_mat_arr).numpy()

In [0]:
# function for the recommendation
def recommend_movie(cosine_sim_data, name):
    movie=[]
    idx = indices[indices == name].index[0]
    sort_index = pd.Series(cosine_sim_data[idx]).sort_values(ascending = False)
    top_10 = sort_index.iloc[1:11]
    for i in top_10.index:
        movie.append(indices[i])
    return movie

In [11]:
tf_cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.04303315, 0.03086067,
        0.03390318],
       [0.        , 1.        , 0.04472136, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04472136, 1.        , ..., 0.04714045, 0.10141851,
        0.07427814],
       ...,
       [0.04303315, 0.        , 0.04714045, ..., 1.        , 0.15936381,
        0.13130643],
       [0.03086067, 0.        , 0.10141851, ..., 0.15936381, 1.        ,
        0.21971769],
       [0.03390318, 0.        , 0.07427814, ..., 0.13130643, 0.21971769,
        1.        ]])

In [12]:
recommend_movie(tf_cosine_sim, 'Transformers Prime')

['Transformers: Robots in Disguise',
 'Pickpockets',
 'Spivak',
 'We Bare Bears',
 'El desconocido',
 'Queer Eye',
 'Men in Black II',
 'We Are Young. We Are Strong.',
 'El señor de los Cielos',
 'Inside the World’s Toughest Prisons']

In [13]:
recommend_movie(tf_cosine_sim, 'Naruto Shippuden : Blood Prison')

['DEATH NOTE',
 'Naruto Shippûden the Movie: Bonds',
 'Mujrim',
 'Naruto the Movie 2: Legend of the Stone of Gelel',
 'Naruto Shippuden: The Movie: The Lost Tower',
 'Naruto Shippûden the Movie: The Will of Fire',
 'The Chef Show',
 "Inuyasha the Movie - L'isola del fuoco scarlatto",
 'Flying Guillotine 2',
 'Naruto the Movie 3: Guardians of the Crescent Moon Kingdom']